In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from typing import Dict, List
import datetime
import json
import matplotlib.pyplot as plt

# 定数

## 挙動の定義

### DATE
ある日付での都道府県ごとの件数を表示する際、日付を指定。
- ファイルの中の最終日を指定する場合はNoneを設定
- 特定の日付を指定する場合は、「YYYY-MM-DD」形式の文字列を設定

In [ ]:
DATE = None
# DATE = '2020-04-05'

### PREF
ある都道府県での日別の件数を表示する際の、都道府県を指定。  
英語表記で指定する。  
使用可能な都道府県は、`prefectures.csv` のファイルを参考に。

In [ ]:
# PREF = ['osaka']
PREF = ['tokyo', 'kanagawa', 'saitama', 'chiba', 'osaka', 'hyogo', 'fukuoka']

### CALC_PREF_FIELD
集計する都道府県の項目  
・"residence_pref" : 居住都道府県  
・"consultation_pref" : 受診都道府県

In [ ]:
# CALC_PREF_FIELD = 'residence_pref'
CALC_PREF_FIELD = 'consultation_pref'

## 定数、変数

In [ ]:
DATA_FILE = 'COVID-19.csv'
PREF_FILE = 'prefectures.csv'

# 都道府県のCSV読み取り、ローマ字名への変換用辞書を作成

In [ ]:
df_pref = pd.read_csv(PREF_FILE, names = ('jp', 'kana', 'en'))

In [ ]:
# [DEBUG] 都道府県CSVの先頭行を表示
# df_pref.head()

In [ ]:
dic_pref_en = {}
for idx, row in df_pref.iterrows():
    dic_pref_en[row['jp']] = row['en']

# 最新のCOVID-19データファイルをダウンロードするには次のセルを有効にして実行する。

In [ ]:
# DATA_URL = 'https://dl.dropboxusercontent.com/s/6mztoeb6xf78g5w/COVID-19.csv'

# import urllib.request

# with urllib.request.urlopen(DATA_URL) as u:
#   with open(DATA_FILE, 'bw') as o:
#     o.write(u.read())

# COVID-19のデータファイルを読み取り

In [ ]:
df = pd.read_csv(DATA_FILE)

In [ ]:
# [DEBUG] CSVの列名を表示
df.columns

In [ ]:
# [DEBUG] CSVの先頭行を表示
# df.head()

In [ ]:
# [DEBUG] CSVの末尾行を表示
# df.tail()

# COVID-19のデータを加工

In [ ]:
for idx, row in df.iterrows():
    # 確定日を日付型にする
    fix_date = None
    if not pd.isnull(row['確定日']):
        fix_date = datetime.datetime.strptime(row['確定日'], '%m/%d/%Y')
    df.at[idx, 'fix_date'] = fix_date
    
    # 都道府県を英語表記にする
    residence_pref = None
    if row['居住都道府県'] in dic_pref_en:
        residence_pref = dic_pref_en[row['居住都道府県']]
    df.at[idx, 'residence_pref'] = residence_pref
    
    # 受診都道府県を英語表記にする。
    consultation_pref = None
    if row['受診都道府県'] in dic_pref_en:
        consultation_pref = dic_pref_en[row['受診都道府県']]
    df.at[idx, 'consultation_pref'] = consultation_pref
    
    # 通しに「id」の別名をつける
    df.at[idx, 'id'] = row['通し']

In [ ]:
# [DEBUG] 加工したデータの先頭行を表示
# df.head()

In [ ]:
# [DEBUG] データの列名を表示
# for col in df.columns:
#     print(col)

# COVID-19のデータから対象データをフィルターする
※対象外のデータを除外する

In [ ]:
df_filter = df

In [ ]:
# 居住地都道府県（英語）がNaNの行を削除する（都道府県以外の国名等が入っているもの）
df_filter = df_filter[df_filter[CALC_PREF_FIELD].notnull()]

In [ ]:
# df[['居住都道府県', 'fix_date', 'ステータス']].head()
# df_filter.groupby('ステータス').count()['id']

# ステータスがNaN（死亡でも退院でもない）の行のみ抽出する
# df_filter = df_filter[df_filter['ステータス'].isnull()]

In [ ]:
# [DEBUG] フィルターしたデータの先頭行を表示
# df_filter.head()

In [ ]:
# [DEBUG] フィルターしたデータの列名を表示
# for col in df_filter.columns:
#     print(col)

# 集計して件数を表示してみる

In [ ]:
# df_filter.groupby('residence_pref').count()['id'].reset_index().sort_values('id', ascending=False)
df_filter.groupby(CALC_PREF_FIELD).count()['id'].reset_index().sort_values('id', ascending=False)

In [ ]:
pd.DataFrame(df_filter.groupby(['fix_date']).count()['id']).sort_values('fix_date')

In [ ]:
# [DEBUG] 指定された日付の全行表示
# date = DATE

# if date is None:
#     max_date = df_filter['fix_date'].max()
#     date = datetime.datetime.strftime(df_filter['fix_date'].max(), '%Y-%m-%d')

# print('query date : %s' % (date,))
# query = 'fix_date == "%s"' % (date,)

# for idx, row in df_filter.query(query).iterrows():
#     print('%d,%s,%s,%s,%s,%s' % (idx,row['id'], row['受診都道府県'], row['residence_pref'], row['居住都道府県'], row['consultation_pref']))

# 件数を集計する

In [ ]:
df_count = pd.DataFrame(df.groupby(['fix_date', CALC_PREF_FIELD]).count()['id']).sort_values([CALC_PREF_FIELD, 'fix_date'])

In [ ]:
df_count = df_count.reset_index()

In [ ]:
df_count.head()

In [ ]:
for idx, row in df_count.iterrows():
    # print(row)
    pass

# グラフ描画

## 都道府県指定、日付ごと

In [ ]:
# query = 'residence_pref in %s' % json.dumps(PREF)
query = 'consultation_pref in %s' % json.dumps(PREF)
df_graph = df_count.query(query)

In [ ]:
# df_graph = df_count.query('residence_pref == "tokyo"')

In [ ]:
df_graph.head()

In [ ]:
# fig, ax = plt.subplots(figsize=(20, 10))

fig, ax = plt.subplots(figsize=(10, 5))
# ax.set_xticklabels('', rotation=90)
plt.xticks(rotation=90)

sns.lineplot(
    ax=ax,
    data=df_graph,
    x='fix_date',
    y='id',
    hue=CALC_PREF_FIELD)

## 日付指定、都道府県ごと

In [ ]:
date = DATE

if date is None:
    date = datetime.datetime.strftime(df_count['fix_date'].max(), '%Y-%m-%d')

In [ ]:
date

In [ ]:
query = 'fix_date == %s' % json.dumps(date)
df_graph = df_count.query(query).sort_values(['id'])

In [ ]:
df_graph.head()

In [ ]:
for idx, row in df_graph.iterrows():
    print(row)

In [ ]:
# fig, ax = plt.subplots(figsize=(20, 10))

fig, ax = plt.subplots(figsize=(10, 5))
# ax.set_xticklabels('', rotation=90)
plt.xticks(rotation=90)

sns.barplot(
    ax=ax,
    data=df_graph,
    x=CALC_PREF_FIELD,
    y='id')